In [1]:
from logging import getLogger
logger = getLogger("main_logger")
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval
from hyperopt.pyll import scope
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV

from auto_lgb import AutoLGB
import seaborn as sns
%matplotlib inline
sns.set(rc={'figure.figsize': [9, 9]}, font_scale=1.2)

%load_ext autoreload
%autoreload 2

Create directory for logs: /Users/mb/Documents/kaggle/machine_learning_template/logs


In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
df["label"] = np.where(df["Transported"] == True, 1, 0)

In [4]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,label
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,1


In [5]:
label = "label"
Y = df[label]
selected_cols = [ 
    "HomePlanet",
    "CryoSleep",
    "Destination",
    "Age",
    "VIP",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck"
]
X = df[selected_cols]

# impute missing data for train data
encoder = {}
for col in X.columns:
    if X[col].dtypes == object:
        X[col].fillna(X[col].mode()[0], inplace=True)
        encoder[col] = LabelEncoder()
        X[col] = encoder[col].fit_transform(X[col])
    else:
        X[col].fillna(X[col].median(), inplace=True)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)


/Users/mb/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/var/folders/tp/ht5m1xnd5fbdky081xlvlsk80000gn/T/ipykernel_92941/4060063806.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = encoder[col].fit_transform(X[col])


In [6]:
#model = AutoLGB(objective="binary", metric="binary_error", n_est=1000, n_hpopt=1000)
model = AutoLGB()
model.tune(X_train, Y_train)
model.fit(X_train, Y_train)

2022-09-16 15:29:13 - main_logger-INFO - [auto_lgb.py:tune:79] Starting optimization process for model LBM


100%|█████████| 10/10 [00:08<00:00,  1.24trial/s, best loss: 0.2031224322103533]

2022-09-16 15:29:21 - main_logger-INFO - [auto_lgb.py:tune:85] Best params for model: {'boosting_type': 'gbdt', 'colsample_bytree': 1.0, 'feature_pre_filter': False, 'learning_rate': 0.02331150906634403, 'max_bin': 158, 'max_depth': 4, 'metric': 'binary_error', 'min_child_samples': 102, 'n_estimators': 269, 'n_jobs': -1, 'num_leaves': 20, 'objective': 'binary', 'reg_alpha': 4.543963035387209, 'reg_lambda': 4.95609193301828, 'seed': 1, 'subsample': 0.7000000000000001, 'subsample_freq': 1, 'verbosity': -1}
2022-09-16 15:29:21 - main_logger-INFO - [auto_lgb.py:tune:86] Time taken to run for: 0.1(mins)
2022-09-16 15:29:21 - main_logger-INFO - [auto_lgb.py:fit:94] Fitting model with input data
2022-09-16 15:29:21 - main_logger-INFO - [auto_lgb.py:fit:98] Model training accuracy: 0.8031224322103533


In [7]:
prediction = model.predict(X_test)

In [8]:
y_pred = (model.predict(X_train) >= 0.5).astype(int)
train_acc = accuracy_score(Y_train, y_pred)
print(f"Accuracy on train set: {train_acc}")
y_pred = (model.predict(X_test) >= 0.5).astype(int)
test_acc = accuracy_score(Y_test, y_pred)
print(f"Accuracy on test set: {test_acc}")

Accuracy on train set: 0.8031224322103533
Accuracy on test set: 0.7906441717791411


In [59]:
# Inference for submission
test_df = pd.read_csv("data/test.csv")
for col in selected_cols:
    if test_df[col].dtypes != object:
        test_df[col].fillna(test_df[col].median(), inplace=True)
    else:
        test_df[col].fillna(test_df[col].mode()[0], inplace=True)
        test_df[col] = encoder[col].transform(test_df[col])

X_pred = test_df[selected_cols]
y_pred = model.predict(X_pred)
test_df["Transported"] = y_pred
test_df["Transported"] = np.where(test_df["Transported"] >= 0.5, True, False)
output_df = test_df[["PassengerId", "Transported"]]
output_df.to_csv("final_submission.csv", index=False)

In [60]:
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,0,1,G/3/S,2,27.0,0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,True
1,0018_01,0,0,F/4/S,2,19.0,0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,False
2,0019_01,1,1,C/0/S,0,31.0,0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,True
3,0021_01,1,0,C/1/S,2,38.0,0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,True
4,0023_01,0,0,F/5/S,2,20.0,0,10.0,0.0,635.0,0.0,0.0,Brence Harperez,True
